# Baseline: AprilTag pose estimation

В этом ноутбуке реализован базовый метод: детекция AprilTag (tag36h11) на изображении, оценка позы в системе координат камеры и преобразование в body-frame FRD для сообщения MAVLink LANDING_TARGET.


In [2]:
import cv2
import numpy as np
from pupil_apriltags import Detector  # pip install pupil-apriltags

FX = 600.0
FY = 600.0
CX = 320.0
CY = 240.0

CAMERA_MATRIX = np.array([[FX, 0, CX],
                          [0, FY, CY],
                          [0,  FY, CY],
                          [0,  0,  1]], dtype=np.float32)

DIST_COEFFS = np.zeros((4, 1), dtype=np.float32)  # пока считаем, что дисторсии нет

TAG_SIZE = 0.16  # 16 см


In [3]:
def create_detector():
    detector = Detector(
        families="tag36h11",
        nthreads=1,
        quad_decimate=1.0,
        quad_sigma=0.0,
        refine_edges=1,
        decode_sharpening=0.25,
        debug=0,
    )
    return detector


In [4]:
def detect_from_image(image_path: str):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"[ERROR] Cannot read image: {image_path}")
        return

    detector = create_detector()

    detections = detector.detect(
        img,
        estimate_tag_pose=True,
        camera_params=[FX, FY, CX, CY],
        tag_size=TAG_SIZE,
    )

    if len(detections) == 0:
        print("[INFO] No tags detected")
        return

    print(f"[INFO] Detected {len(detections)} tag(s)")

    for det in detections:
        tvec = det.pose_t.reshape(3)

        print("Pose in camera frame (meters):")
        print(f"  x: {tvec[0]:.3f}, y: {tvec[1]:.3f}, z: {tvec[2]:.3f}")

        forward = tvec[2]
        right = tvec[0]
        down = tvec[1]

        print("Pose in body-frame FRD (for LANDINGTARGET):")
        print(f"  forward: {forward:.3f} m")
        print(f"  right:   {right:.3f} m")
        print(f"  down:    {down:.3f} m")
        print("-----")


In [6]:
detect_from_image("../tag36h11-0.png")  # путь к твоему изображению с тегом


[INFO] Detected 1 tag(s)
Pose in camera frame (meters):
  x: -0.100, y: -0.050, z: 0.375
Pose in body-frame FRD (for LANDINGTARGET):
  forward: 0.375 m
  right:   -0.100 m
  down:    -0.050 m
-----
